In [1]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393142 sha256=123e75c70e617f87f065d58a1d61514e6a6ec8dd29945a894845fc418f24bf7b
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [10]:
#importing libraries
import fasttext as ft
import pandas as pd
import numpy as np
import re

In [11]:
df = pd.read_csv('/content/Language Detection.csv', names=['text', 'lang'])

In [12]:
df = df.reindex(np.random.permutation(df.index)).reset_index(drop=True)

In [13]:
df[:3]

,text,lang
0,எனக்கு தெரியாது என்று பயப்படுகிறேன்.,Tamil
1,Θέλω να συνεχίσω δεν είμαι κουρασμένος από αυτ...,Greek
2,man ist überflutet überflutet bedeutet ertrink...,German


In [15]:
def clean_text(row):
  cleaned = re.sub(r'<.*?>', '', row['text']).lower().strip()
  return cleaned

In [16]:
df['text'] = df.apply( lambda row: clean_text(row), axis=1 )

In [18]:
df.tail()

,text,lang
10333,που σημαίνει που ωθεί το άτομο να επαναλάβει ξ...,Greek
10334,eu também vou te dar uma dica inteligente para...,Portugeese
10335,hoe gaat het?,Dutch
10336,(δείτε την γρήγορη διαγραφή σελίδων για πιθανο...,Greek
10337,[110]​ la política que se encarga de combatir ...,Spanish


In [24]:
10338 * 75 // 100

7753

In [20]:
df_new = df[['lang','text']]

In [21]:
df_new.head()

,lang,text
0,Tamil,எனக்கு தெரியாது என்று பயப்படுகிறேன்.
1,Greek,θέλω να συνεχίσω δεν είμαι κουρασμένος από αυτ...
2,German,man ist überflutet überflutet bedeutet ertrink...
3,Italian,la discesa dei prezzi per l'hardware e lo svil...
4,French,"depuis peu, et au niveau international, ils te..."


In [22]:
def normalize_text(row):
    
    label = '__label__' + str(row['lang'])
    txt = str(row['text'])
    
    return ' '.join(( label + ' , ' + txt ).split())

In [23]:
df_new['normalized'] = df_new.apply( lambda row: normalize_text(row), axis=1 )

<ipython-input-23-d48f00451101>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['normalized'] = df_new.apply( lambda row: normalize_text(row), axis=1 )


In [25]:
SPLIT = 7753
train = df_new['normalized'][:SPLIT].copy()
test = df_new['normalized'][SPLIT:].copy()

In [28]:
np.savetxt('lang_train.txt', train.values, fmt="%s")
np.savetxt('lang_test.txt', test.values, fmt="%s")

In [29]:
# Training
model = ft.train_supervised('lang_train.txt', lr=0.1, epoch=25)

In [31]:
# Evaluation
result = model.test('lang_test.txt')

In [32]:
result

(2585, 0.9535783365570599, 0.9535783365570599)

In [33]:
model.save_model('language_detection_model.bin')

In [35]:
# Load the model
loaded_model = ft.load_model('language_detection_model.bin')

In [36]:
# Predict language
text = "This is a sample text"
predicted_language = loaded_model.predict(text)
print(predicted_language)

(('__label__English',), array([0.99706268]))


In [37]:
text = 'எனக்கு தெரியாது என்று பயப்படுகிறேன்.'
predicted_language = loaded_model.predict(text)
print(predicted_language)

(('__label__Tamil',), array([0.99997497]))
